In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
import pathlib
import pickle
from pathlib import Path
from skimage.io import imread
import seaborn as sns
sns.set_style('white')
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record

In [4]:
model_path = Path('/mnt/sda2/SMLAT/training_runs/good_models/model_8865_venus.pkl')

In [5]:
with open(model_path, 'rb') as f:
    chromo_model = pickle.load(f)

In [6]:
chromo_model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [7]:
plot_train_record(chromo_model)

#### Photon count range, get background average from simulation using model paramteres

In [8]:
mol_photons = (chromo_model.data_generator.simulation_params['min_photon'] + 1) /2 * chromo_model.data_generator.psf_params['photon_scale']

In [9]:
mol_photons

2100.0

#### Plot network outputs for the eval images after tiling and re-tiling of one eval image

In [32]:
from cellbgnet.analyze_eval import recognition, plot_full_img_predictions, assemble_full_img_predictions

In [11]:
eval_img = chromo_model.evaluation_params['eval_imgs'][10][np.newaxis,:]

In [12]:
plt.figure()
plt.imshow(eval_img[0], cmap='gray')
plt.colorbar()
plt.show()

In [13]:
fov_size = [eval_img.shape[2] * 65, eval_img.shape[1] * 65]

In [14]:
fov_size

[84630, 67665]

In [15]:
preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=eval_img,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.3,
                                             nms_threshold=0.7, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]


In [16]:
plot_full_img_predictions(chromo_model, plot_infs=plot_data, eval_csv=None, plot_num=1, fov_size=fov_size, pixel_size=[65, 65])

#### Testing real image

In [33]:
images_dir = Path('/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_8865/venus/')
img_filenames = sorted(list(images_dir.glob('*.tiff')))

In [61]:
img_path = Path('/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_8865/venus/Pos01_img_000000110.tiff')

In [62]:
real_img = imread(img_path)[np.newaxis, :]


In [63]:
preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=real_img,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.3,
                                             nms_threshold=0.7, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]


In [64]:
plot_full_img_predictions(chromo_model, plot_infs=plot_data, eval_csv=None, plot_num=1,
                        fov_size=fov_size, pixel_size=[65, 65])

In [65]:
len(preds_tmp)

229

In [66]:
img_infs = assemble_full_img_predictions(chromo_model, plot_data)

In [67]:
img_infs.keys()

dict_keys(['Probs', 'XO', 'YO', 'ZO', 'Int', 'BG', 'XO_sig', 'YO_sig', 'ZO_sig', 'Int_sig', 'Probs_ps', 'XO_ps', 'YO_ps', 'ZO_ps', 'Samples_ps', 'raw_img', 'only_bg'])

In [68]:
for key, value in img_infs.items():
    plt.figure()
    plt.imshow(value)
    plt.colorbar()
    plt.title(key)
    plt.show()

/tmp/ipykernel_5661/3544528225.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


In [69]:
plt.figure()
plt.imshow(img_infs['BG'])
plt.colorbar()
plt.title('Predicted PSF')
plt.show()

In [70]:
plt.figure()
plt.imshow(img_infs['only_bg'][:-200, :-200])
plt.colorbar()
plt.title('Only bg')
plt.show()

##### Loop over all images and plot variability in time

In [22]:
number_preds = []
for img_path in img_filenames:
    real_img = imread(img_path)[np.newaxis, :]
    preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=real_img,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.1,
                                             nms_threshold=0.7, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])
    number_preds.append(len(preds_tmp))
    print(img_path)

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000000.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000001.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000002.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000003.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000038.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000039.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000040.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000041.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000076.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000077.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000078.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000079.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000114.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000115.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000116.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000117.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000152.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000153.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000154.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000155.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

...

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000418.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000419.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000420.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000421.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000456.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000457.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000458.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000459.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000494.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000495.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000496.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000497.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000532.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000533.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000534.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000535.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000570.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000571.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000572.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000573.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

In [25]:
plt.figure()
plt.plot(np.arange(0, len(number_preds)), number_preds)
plt.xlabel('Frame number')
plt.ylabel('Number of dots predicted')
plt.title('Chromosome dots 8865 Pos01')
plt.show()